<a href="https://colab.research.google.com/github/RbsFaneva/Quant-Trading/blob/main/bot_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# Data and analysis libraries
import polars as pl                         # Data frames for financial data
import numpy as np                          # Numerical computing library
from datetime import datetime, timedelta    # Date and time operation
import random

# Machine learning libraries
import torch                                # PyTorch framwork
import torch.nn as nn                       # Neural network modules
import torch.optim as optim                 # Optimization algorithms
import research

In [38]:
!git push origin

fatal: could not read Username for 'https://github.com': No such device or address
